# Análise de Portfólio
Análise de portfólio de Mínimo Risco com técnicas de VaR (Value at Risk), otimização por simulações.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import yfinance as yf
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Pesos = {"ITUB4.SA":0.10, "CPLE3.SA":0.20, "LREN3.SA":0.20, "EMBR3.SA":0.10, "PSSA3.SA":0.10, "GOAU4.SA":0.30}

Periodo_Inicio = datetime.datetime(2016,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
# DadosAcoes = pdr.get_data_yahoo(list(Pesos.keys()), start=Periodo_Inicio, end=Periodo_Termino)
DadosAcoes = yf.download(list(Pesos.keys()), start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados = DadosAcoes["Adj Close"]

In [ ]:
Dados

### Indicadores de Portfólio e Risco Máximo Esperado
Calcula indicadores de retorno esperado e risco com uso de Value at Risk.

- Retorno dos Ativos

In [ ]:
RetornoDosAtivos = Dados.pct_change().fillna(0)

In [ ]:
RetornoDosAtivos

- Retorno do Portfólio

In [ ]:
RetornoDosAtivos["Portfolio"] = RetornoDosAtivos.dot(list(Pesos.values()))

In [ ]:
RetornoDosAtivos

- Retorno Médio Diário Esperado

In [ ]:
RetornoMedioDiario = RetornoDosAtivos["Portfolio"].mean()

In [ ]:
print("Retorno Médio Diário do Portfólio: {0:.2%}".format(RetornoMedioDiario))

- Retorno Médio Mensal Esperado

In [ ]:
RetornoMedioMensal = Dados.resample("M").last().pct_change().dot(list(Pesos.values())).mean()

In [ ]:
print("Retorno Médio Mensal do Portfólio: {0:.2%}".format(RetornoMedioMensal))

- Retorno Médio Anualizado Esperado

In [ ]:
RetornoMedioAnualizado = Dados.resample("Y").last().pct_change().dot(list(Pesos.values())).mean()

In [ ]:
print("Retorno Médio Anualizado do Portfólio: {0:.2%}".format(RetornoMedioAnualizado))

- Risco do Portfólio pelo Desvio

In [ ]:
RiscoPorDesvio = RetornoDosAtivos["Portfolio"].std()

In [ ]:
RiscoPorDesvioDiario = RiscoPorDesvio
RiscoPorDesvioMensal = RiscoPorDesvio * np.sqrt(21)
RiscoPorDesvioAnualizado = RiscoPorDesvio * np.sqrt(252)

In [ ]:
print("Risco Diário do Portfólio: {0:.2%}".format(RiscoPorDesvioDiario))
print("Risco Mensal do Portfólio: {0:.2%}".format(RiscoPorDesvioMensal))
print("Risco Anualizado do Portfólio: {0:.2%}".format(RiscoPorDesvioAnualizado))

- Riscos do Portfólio com Value at Risk Histórico

In [ ]:
RiscoPorVarHistorico = np.quantile(RetornoDosAtivos["Portfolio"], 0.05)

In [ ]:
RiscoPorVarHistoricoDiario = RiscoPorVarHistorico
RiscoPorVarHistoricoMensal = RiscoPorVarHistorico * np.sqrt(21)
RiscoPorVarHistoricoAnualizado = RiscoPorVarHistorico * np.sqrt(252)

In [ ]:
print("Risco Diário do Portfólio: {0:.2%}".format(RiscoPorVarHistoricoDiario))
print("Risco Mensal do Portfólio: {0:.2%}".format(RiscoPorVarHistoricoMensal))
print("Risco Anualizado do Portfólio: {0:.2%}".format(RiscoPorVarHistoricoAnualizado))

- Riscos do Portfólio com Value at Risk Paramétrico

In [ ]:
RiscoPorVarParam = RetornoMedioDiario - norm.ppf(0.95) * RiscoPorDesvio

In [ ]:
RiscoPorVarParamDiario = RiscoPorVarParam
RiscoPorVarParamMensal = RiscoPorVarParam * np.sqrt(21)
RiscoPorVarParamAnualizado = RiscoPorVarParam * np.sqrt(252)

In [ ]:
print("Risco Diário do Portfólio: {0:.2%}".format(RiscoPorVarParamDiario))
print("Risco Mensal do Portfólio: {0:.2%}".format(RiscoPorVarParamMensal))
print("Risco Anualizado do Portfólio: {0:.2%}".format(RiscoPorVarParamAnualizado))

### Simulação de Portfólios
Simula um número de portfólios baseado nos seguintes parâmetros:
- Ativos: Dataframe com Ativos e seus preços.
- NumPortfolios: Número de portfólios à simular.

In [ ]:
def SimularPortfolios(Ativos, NumPortfolios):
    RetornosSimulados = []
    RiscosSimulados = []
    VaR_Hist_Simulados = []
    VaR_Param_Simulados = []
    PesosSimulados = []
    NumAtivos = len(Ativos.columns)
    
    # Retorno dos Ativos
    Ativos = Ativos.pct_change().fillna(0)

    # Simulações
    for portfolio in range(NumPortfolios):

        # Simula pesos para um portfólio
        Pesos = np.random.random(NumAtivos)
        Pesos = Pesos/np.sum(Pesos)
        PesosSimulados.append(Pesos)
        
        # Retorno do Portfólio simulado
        Retornos = Ativos.dot(Pesos).values
        Retorno = Retornos.mean()
        RetornosSimulados.append(Retorno)
        
        # Risco do portfólio simulado
        Risco = Retornos.std()
        VaR_Hist = np.quantile(Retornos, 0.05)
        VaR_Param = Retornos.mean() - norm.ppf(0.95) * Retornos.std()
       
        RiscosSimulados.append(Risco)
        VaR_Hist_Simulados.append(VaR_Hist)
        VaR_Param_Simulados.append(VaR_Param)

    # Dataframe dos Portfólios simulados
    Simulacoes = {"Retorno":RetornosSimulados, "Risco":RiscosSimulados, "VaR_Hist":VaR_Hist_Simulados, "VaR_Param":VaR_Param_Simulados}
    for counter, symbol in enumerate(Ativos.columns.tolist()):
        Simulacoes[symbol] = [w[counter] for w in PesosSimulados]
    
    Portfolios = pd.DataFrame(Simulacoes)
    return(Portfolios)

In [ ]:
Portfolios = SimularPortfolios(Dados, 10000)

In [ ]:
Portfolios

- Performance de cada Portfólio

In [ ]:
Portfolios["Sharpe"] = Portfolios["Retorno"] / Portfolios["Risco"]

In [ ]:
Portfolios

### Fronteira Eficiente com VaR
Análise de posicionamento dos portfólios simulados.

- Portfólio de Maior Retorno

In [ ]:
PortfolioMaiorRetorno = Portfolios.iloc[Portfolios["Retorno"].idxmax()]

In [ ]:
PortfolioMaiorRetorno

- Portfólio de Menor Risco

In [ ]:
PortfolioMenorRisco = Portfolios.iloc[Portfolios["Risco"].idxmin()]

In [ ]:
PortfolioMenorRisco

- Portfólio de Melhor Performance

In [ ]:
PortfolioMelhorSharpe = Portfolios.iloc[Portfolios["Sharpe"].idxmax()]

In [ ]:
PortfolioMelhorSharpe

- Fronteira Eficiente usando Risco vs Retorno

In [ ]:
FronteiraEficiente = plg.Figure()

FronteiraEficiente.add_trace(plg.Scatter(x=Portfolios["Risco"],
                                         y=Portfolios["Retorno"],
                                         mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                         name="Portfolios Simulados"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMenorRisco["Risco"]],
                                         y=[PortfolioMenorRisco["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor Risco"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMaiorRetorno["Risco"]],
                                         y=[PortfolioMaiorRetorno["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Retorno"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorSharpe["Risco"]],
                                         y=[PortfolioMelhorSharpe["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Sharpe"))
FronteiraEficiente.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
Plot_Show(FronteiraEficiente, "Portfólios Simulados")

- Portfólio de Menor VaR Histórico

In [ ]:
PortfolioMelhorVarH = Portfolios.iloc[Portfolios["VaR_Hist"].idxmax()]

In [ ]:
PortfolioMelhorVarH

- Portfólio de Menor VaR Paramétrico

In [ ]:
PortfolioMelhorVarP = Portfolios.iloc[Portfolios["VaR_Param"].idxmax()]

In [ ]:
PortfolioMelhorVarP

- Fronteira Eficiente usando VaR Histórico vs Retorno

In [ ]:
FronteiraEficiente = plg.Figure()

FronteiraEficiente.add_trace(plg.Scatter(x=Portfolios["VaR_Hist"],
                                         y=Portfolios["Retorno"],
                                         mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                         name="Portfolios Simulados"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMenorRisco["VaR_Hist"]],
                                         y=[PortfolioMenorRisco["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor Risco"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorVarH["VaR_Hist"]],
                                         y=[PortfolioMelhorVarH["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor VaR HIstórico"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorVarP["VaR_Hist"]],
                                         y=[PortfolioMelhorVarP["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor VaR Paramétrico"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMaiorRetorno["VaR_Hist"]],
                                         y=[PortfolioMaiorRetorno["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Retorno"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorSharpe["VaR_Hist"]],
                                         y=[PortfolioMelhorSharpe["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Sharpe"))
FronteiraEficiente.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
Plot_Show(FronteiraEficiente, "Portfólios Simulados")

- Fronteira Eficiente usando VaR Paramétrico vs Retorno

In [ ]:
FronteiraEficiente = plg.Figure()

FronteiraEficiente.add_trace(plg.Scatter(x=Portfolios["VaR_Param"],
                                         y=Portfolios["Retorno"],
                                         mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                         name="Portfolios Simulados"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMenorRisco["VaR_Param"]],
                                         y=[PortfolioMenorRisco["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor Risco"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorVarH["VaR_Param"]],
                                         y=[PortfolioMelhorVarH["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor VaR HIstórico"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorVarP["VaR_Param"]],
                                         y=[PortfolioMelhorVarP["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor VaR Paramétrico"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMaiorRetorno["VaR_Param"]],
                                         y=[PortfolioMaiorRetorno["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Retorno"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorSharpe["VaR_Param"]],
                                         y=[PortfolioMelhorSharpe["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Sharpe"))
FronteiraEficiente.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
Plot_Show(FronteiraEficiente, "Portfólios Simulados")